In [ ]:
import warnings
# warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

import pandas as pd
import numpy as np



# **加载数据**

In [ ]:
#解压缩
!unzip /kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
!unzip /kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
!unzip /kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip

In [ ]:
train=pd.read_csv('/kaggle/working/labeledTrainData.tsv',delimiter='\t',quoting=3)
test=pd.read_csv('/kaggle/working/testData.tsv',delimiter='\t',quoting=3)
unlabeled_train=pd.read_csv("/kaggle/working/unlabeledTrainData.tsv",
                            header=0,    #设置导入dataframe的列名称，默认为infer
                            delimiter='\t',
                            quoting=3)   #可以如实打印csv中内容，
                                         #不设置该参数引号内内容会读取错误

In [ ]:
print(train.shape)
print(test.shape)
print(unlabeled_train.shape)
print(train.info)

print(train['review'].size)
print(test['review'].size)
print(unlabeled_train['review'].size) 
# pd.isnull(train)

In [ ]:
#bs4库 是解析、遍历、维护、“标签树“的功能库(bs4库把html源代码重新进行了格式化,
#                                      从而方便我们对其中的节点、标签、属性等进行操作)
!pip3 install Beautifulsoup4

In [ ]:
%%writefile Word2VecUtil.py
import re
import nltk
#nltk是一个nlp的工具包，内含数据集、python模块等。
#可搜索文本，计数词汇

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup      #最主要功能就是从网页抓取数据
from nltk.corpus import stopwords  #导入英语停用词，如at of you a that等
from nltk.stem.snowball import SnowballStemmer #一个小的语言转换库 支持15种语言

from multiprocessing import Pool

class KaggleWord2VecUtility(object):

    @staticmethod
    #删除停用词，返回list
    def review_to_wordlist(review, remove_stopwords=False): 
        # 1. HTML 删除
        review_text = BeautifulSoup(review, "html.parser").get_text() #创建BS对象并获取其文本
        # 2. 将特殊字符转为空格 sub(pattern,repl,string)
        # pattern：正则中的模式字符串 string：被处理的字符串
        review_text = re.sub('[^a-zA-Z]', ' ', review_text)
        # 3. 转化为小写+分割
        words = review_text.lower().split()
        # 4. 删除停用词
        if remove_stopwords:
            #创建一个无序不重复元素集（即停用词集合）
            stops = set(stopwords.words('english')) 
            words = [w for w in words if not w in stops]
        # 5. 提取词干
        stemmer = SnowballStemmer('english')  #选择语言为english
        words = [stemmer.stem(w) for w in words] #提取每个词语的主干
        # 6. 返回一个list
        return(words)

    @staticmethod
    #用join将word用“ ”连接起来
    def review_to_join_words( review, remove_stopwords=False ):
        words = KaggleWord2VecUtility.review_to_wordlist(\
            review, remove_stopwords=False)
        join_words = ' '.join(words)
        return join_words

    @staticmethod
    def review_to_sentences( review, remove_stopwords=False ):
        #下行是一个将英文文本划分成句子的模型，划分依据是每个句子结束后留有空格
        tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        # 1. nltk tokenize分割句子为单词，删除空格strip()
        raw_sentences = tokenizer.tokenize(review.strip())
        # 2. 遍历每个句子
        sentences = []
        for raw_sentence in raw_sentences:
            # 为空则skip
            if len(raw_sentence) > 0:
                # 删除标签，用空格替换非字母字符，删除停用词
                sentences.append(\
                    KaggleWord2VecUtility.review_to_wordlist(\
                    raw_sentence, remove_stopwords))
        return sentences


    # 参考 : https://gist.github.com/yong27/7869662
    # http://www.racketracer.com/2016/07/06/pandas-in-parallel/
    # 与多个线程一起使用以提高速度
    @staticmethod
    def _apply_df(args):
        df, func, kwargs = args
        return df.apply(func, **kwargs)

    @staticmethod
    def apply_by_multiprocessing(df, func, **kwargs):
        # 获取关键字项中的worker参数
        workers = kwargs.pop('workers')
        # 定义Pool进程池
        pool = Pool(processes=workers)
        #map()根据提供的函数对指定序列做映射 
        result = pool.map(KaggleWord2VecUtility._apply_df, [(d, func, kwargs)
                for d in np.array_split(df, workers)])
        pool.close()
        # 작업 결과를 합쳐서 반환
        return pd.concat(result)

In [ ]:
# 修改word2vecutil文件
# %load Word2VecUtil.py

In [ ]:
from Word2VecUtil import KaggleWord2VecUtility

In [ ]:
print(train['review'][0][:50])
KaggleWord2VecUtility.review_to_wordlist(train['review'][0][:50])
#提取词干了 如going-go

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
sentences = []
for review in train['review']:
    sentences += KaggleWord2VecUtility.review_to_sentences(review,
                                                           remove_stopwords=False)


In [ ]:
for review in unlabeled_train['review']:
    sentences += KaggleWord2VecUtility.review_to_sentences(
        review,remove_stopwords=False)

In [ ]:
# len(sentences)

In [ ]:
print(sentences[0][:10])
print(sentences[1][:10])

# **Word2Vec算法** 
# 通过预处理解析出的句子列表来训练模型

体系结构：“体系结构”选项是skip-gram（默认）或CBOW模型。 skip-gram速度较慢，但效果更好
学习算法：分层softmax（默认）或负采样。默认值在这里工作正常。
对经常出现的单词进行下采样：Google文档建议在0.00001和0.001之间的值。此处，显示接近0.001的值可提高最终模型的准确性。
字向量维：使用许多功能并不总是很好，但通常是更好的模型。合理的值可以从数十到数百，此处指定为300。
上下文/窗口大小：上下文中学习算法应考虑多少个单词？较大的数字对分层softmax有益，但10则很好
辅助线程：要运行的并行进程数，这在计算机之间是不同的，但在大多数系统上使用的值为4到6之间。
最小单词数：有助于将词汇表的大小限制为有意义的单词数。在所有文档中没有多次出现的单词将被忽略。适当的范围是10到100，
因为每个电影都有30条评论，因此最小字数设置为40，以避免过于重视单个电影标题。结果，总词汇量约为15,000个单词。较高的值有助于有限的执行时间。

In [ ]:
import logging
logging.basicConfig(  # 为日志配置基本信息
    format='%(asctime)s : %(levelname)s : %(message)s', #日志输出格式
    level=logging.INFO) # 设置日志级别 低于该级别的日志消息将被忽略
                        # CRITICAL>ERROR>WARNNING>INFO>DEBUG>NOTSET
    

In [ ]:
num_features = 300 # 字符数矢量维
min_word_count = 40 # 最小字符数
num_workers = 4 # 并行处理线程数
context = 10 # 字符串窗口大小
downsampling = 1e-3

#初始化和模型训练
from gensim.models import word2vec
#gensim是个nlp工具包，内含多个常见模型 LSI LDA HDP DTM DIM TF-IDF word2vec paragraph2vec

model = word2vec.Word2Vec(sentences,
                         workers=num_workers,
#                          size=num_features,
                         min_count=min_word_count,
                         window=context,
                         sample=downsampling) #高频词随机下采样的配置阈值，范围（0，1e-5）
model

In [ ]:
#  训练完后删除不必要的内存
model.init_sims(replace=True)

model_name = '300features_40minwords_10text'

model.save(model_name)

# **探索模型结果**

In [ ]:
#提取没有相似性的单词
model.wv.doesnt_match('man woman child kitchen'.split())

In [ ]:
model.wv.doesnt_match('france england germany berlin'.split())

In [ ]:
#提取相似的词
model.wv.most_similar('man')

In [ ]:
model.wv.most_similar('queen')

In [ ]:
# model.wv.most_similar('awful')

In [ ]:
model.wv.most_similar('film')

In [ ]:
# model.wv.most_similar('happy')

In [ ]:
model.wv.most_similar('happi')

通过t-SNE可视化Word2Vec矢量化的单词

In [ ]:
from sklearn.manifold import TSNE  #数据降维和可视化
#TSNE使高于二维的聚类结果以二维方式展现出来
import matplotlib as mpl
import matplotlib.pyplot as plt
import gensim
#gensim从原始的非结构化文本中，无监督学习到文本隐层的主题向量表达
#支持TF-IDF，LSA，LDA，word2vec
import gensim.models as g

In [ ]:
# 用来显示中文标签
# plt.rcParams['font.sans-serif']=['SimHei']
#用于显示负号
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
model_name = '300features_40minwords_10text'
model = g.Doc2Vec.load(model_name)

#model.wv.vocab 直接调用生成的词向量
vocab = list(model.wv.key_to_index)  

X = model[vocab]


# rock_idx = model.wv.key_to_index["rock"]
# vocab = (model.wv.get_vecattr(,))

# 

print(len(X))
# print(X[0][:10])
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X[:100, :])


In [ ]:
df = pd.DataFrame(X_tsne, index=vocab[:100], columns=['x', 'y'])
df.shape 
df.head(10)

In [ ]:
fig = plt.figure()
fig.set_size_inches(40,20)
ax = fig.add_subplot(111)
ax.scatter(df['x'],df['y'])
for word,pos in df.iterrows():
    ax.annotate(word,pos,fontsize=30)
plt.show()

In [ ]:
import numpy as np

def makeFeatureVec(words, model, num_features):
    '''
    在IMDB中每段评论长度不一，因此需要先将每个独立的单词向量转换成等长的特征集合
    对所有单词向量求平均
    '''
    # 初始化一个数组（为了提高速度）
    featureVec = np.zeros((num_features,), dtype='float32')
    
    nwords = 0.
    # Index2word是模型词汇表中单词名称的列表（为了提速）
    index2word_set = set(model.wv.index2word)
    
    # 循环遍历评论中的每个单词，如果它在模型的词汇表中，
    # 则将其及其特征向量循环到总和
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
    
    # 做平均运算
    featureVec = np.divide(featureVec, nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    # 给定一组评论（每个评论一个单词列表），计算每个特征向量的平均特征向量，
    #并返回一个2D numpy数组
     
    counter = 0

    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
       
    for review in reviews:
       if counter%1000 == 0:
           print "Review %d of %d" % (counter, len(reviews))
       
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)

       counter = counter + 1
    return reviewFeatureVecs

def getCleanReviews(reviews):
    clean_reviews = []
    clean_reviews = KaggleWord2VecUtility.apply_by_multiprocessing(\
            reviews['review'], KaggleWord2VecUtility.review_to_wordlist,\
            workers=4)
    return clean_reviews

In [ ]:
%time
trainDataVecs = getAvgFeatureVecs(\
    getCleanReviews(train), model, num_features)

In [ ]:
%time
testDataVecs = getAvgFeatureVecs(\
    getCleanReviews(test), model, num_features)

# **RandomForest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(
    n_estimators=100, n_jobs=-1, random_state=2018)

In [ ]:
%time
forest = forest.fit(trainDataVecs, train['sentiment'])

In [ ]:
from sklearn.model_selection import cross_val_score
%time
score = np.mean(cross_val_score(\
        forest, trainDataVecs, \
        train['sentiment'], cv=10, scoring='roc_auc'))

In [ ]:
score

In [ ]:
result = forest.predict(testDataVecs)

In [ ]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.to_csv('Word2Vec_AverageVEctors_{0:.5f}.csv'.format(score),
             index=False, quoting=3)

In [ ]:
output_sentiment = output['sentiment'].value_counts()
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

In [ ]:
import seaborn as sns
%matplotlib inline

fig, axes = plt.subplots(ncols=2)
fig.set_size_inches(12,5)
sns.countplot(train['sentiment'], ax=axes[0])
sns.countplot(output['sentiment'], ax=axes[1])